# Document the Data Sources
* [Coingecko python code](https://github.com/man-c/pycoingecko)

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Initialize classes

In [30]:
import os
import sys
import configparser
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from wrangler.modules.assets.etl import cryptoMCExtractor as mcap
from utils.modules.etl.loader import sparkDBwls as spark
from utils.modules.etl.loader import sparkNoSQLwls as nosql

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    mcap = importlib.reload(mcap)
    nosql = importlib.reload(nosql)
    spark = importlib.reload(spark)
    
__desc__ = "get crypto macket capitalization data"
clsNoSQL = nosql.NoSQLWorkLoads(desc=__desc__)
# clsSpark = spark.SQLWorkLoads(desc=__desc__)
''' optional - if not specified class will use the default values '''
# prop_kwargs = {"WRITE_TO_TMP":True,   # necessary to emulate the etl dag
#               }
clsMC = mcap.CryptoMarkets(desc=__desc__)
print("\nClass initialization and load complete!")

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All assets-module etl-packages in function-CryptoMarket imported successfully!
All functional SPARKNOSQLWLS-libraries in LOADER-package of ETL-module imported successfully!
All functional SPARKDBWLS-libraries in LOADER-package of ETL-module imported successfully!
sparkNoSQLwls Class initialization complete
sparkNoSQLwls Class initialization complete
FileWorkLoads Class initialization complete
CryptoMarket Class initialization complete

Class initialization and load complete!


## Transform collections into Spark DataFrame

In [31]:
# _data_owner='coingecko'
__SOURCEDBNAME__ = 'tip-historic-marketcap'
__DESTINDBNAME__ = 'tip'
__DESTINTABLE__ = 'mcap_past'

_from_dt=date(2023,3,1)
_to_dt=date(2023,3,31)
_kwargs = {
#     "SOURCEDBNAME":'tip-historic-marketcap',
    "DESTINTBLNAME":'mcap_past',
    "DBAUTHSOURCE":'tip-historic-marketcap',
#     "COLLLIST":['coingecko.2022-04-01.btc','coingecko.2022-04-01.etc'],
#     "COLLLIST":['coingecko.2022-07-01.axial'],
   "HASINNAME":['coingecko','2023-03-01'],
#     "FIND":{'symbol':{'$eq':'btc'}},   # use the find key to define a filter
    "COLUMNSMAP":{'_id':'uuid',
                  'source':'data_source',
                  'name':'asset_name',
                  'id':'alt_asset_id',
                  'symbol':'asset_symbol',
                  'currency' : 'currency',
                  'price.date':'price_date',
                  'price.value':'price_value',
                  'mcap.date':'mcap_date',
                  'mcap.value':'mcap_value',
                  'volume.date':'volume_date',
                  'volume.size':'volume_size',
#                   'audit.mod.by':'created_by',
#                   'audit.mod.date':'created_dt',
                 }
}
_data = clsMC.nosql_to_sql(
    source_db=__SOURCEDBNAME__,
#     coll_list=[],
    destin_db=__DESTINDBNAME__,
    table_name=__DESTINTABLE__,
    **_kwargs,
)

retrieving collections ...


23/04/26 14:07:20 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 128299 ms exceeds timeout 120000 ms
23/04/26 14:07:20 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1053)
	at org.apache.spark.e

23/04/26 14:37:13 ERROR Executor: Exception in task 0.0 in stage 29192.0 (TID 20069)
java.sql.BatchUpdateException: Batch entry 0 INSERT INTO warehouse.mcap_past ("uuid","data_source","alt_asset_id","asset_symbol","asset_name","mcap_value","mcap_date","currency","price_value","price_date","volume_size","volume_date","created_proc","created_dt","created_by") VALUES ('64476a6214c8ee91f463884a','coingecko','phoenix-protocol-b7a9513c-36e9-4a6b-b6ae-6a1a76bb913e','pp','phoenix_protocol',548489.0275745493,'2023-03-02 00:00:00+08'::timestamp,'usd',0.013289691884763082,'2023-03-02 00:00:00+08'::timestamp,5665.271138273307,'2023-03-02 00:00:00+08'::timestamp,'wrangler_assets_etl_CryptoMarket function <nosql_to_sql>','2023-04-26 14:37:13.072953+08'::timestamp,'farmraider') was aborted: ERROR: value too long for type character varying(50)  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:165)
	at org.postgresql.

In [32]:
3379*30

101370

## Spark MongoDB experiment

In [51]:
from pyspark.sql import SparkSession
_appName = " ".join(
    [clsNoSQL.__app__,
     clsNoSQL.__name__,
     clsNoSQL.__package__,
     clsNoSQL.__module__
    ])   #"PySpark MongoDB Examples"
_master = clsNoSQL.sparkMaster  # "local[1]"
_h_ip = clsNoSQL.dbHostIP #"127.0.0.1"
_type = clsNoSQL.dbType   # mongodb
_port = clsNoSQL.dbPort   # 27017
_format = clsNoSQL.dbFormat # mongo
_user = clsNoSQL.dbUser   # "farmraider"
_pswd = clsNoSQL.dbPswd   # "spirittribe"
_auth = clsNoSQL.dbAuthSource   #"tip-historic-marketcap"
_db = clsNoSQL.dbName     # "tip-historic-marketcap"
_coll = 'coingecko.2022-07-01.btc'

_inp_uri = f"{_type}://{_user}:{_pswd}@{_h_ip}/{_db}.{_coll}?authSource={_auth}"
_out_uri = f"{_type}://{_user}:{_pswd}@{_h_ip}/{_db}.{_coll}?authSource={_auth}"

# Create Spark session
spark = SparkSession.builder \
    .appName(_appName) \
    .master(_master) \
    .config("spark.mongodb.input.uri", _inp_uri) \
    .config("spark.mongodb.output.uri", _out_uri) \
    .getOrCreate()

# Read data from MongoDB
df = spark.read.format(_format).load()
df.printSchema()
df.show()

22/12/16 15:15:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- marketcap: double (nullable = true)
 |-- source: string (nullable = true)
 |-- symbol: string (nullable = true)

+--------------------+-------------------+--------------------+---------+------+
|                 _id|               date|           marketcap|   source|symbol|
+--------------------+-------------------+--------------------+---------+------+
|{6390a3bf9d51d344...|2022-07-01 16:00:00|3.659842243234527...|coingecko|   btc|
|{6390a3bf9d51d344...|2022-07-02 16:00:00|3.709165299187163E11|coingecko|   btc|
|{6390a3bf9d51d344...|2022-07-03 16:00:00| 3.67706689137568E11|coingecko|   btc|
|{6390a3bf9d51d344...|2022-07-04 16:00:00|3.685493641350522...|coingecko|   btc|
|{6390a3bf9d51d344...|2022-07-05 16:00:00|3.864251762627784...|coingecko|   